In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# API KEY : 코드 제출을 위해 환경변수 설정
load_dotenv()
sola_api_key = os.getenv('sola_api_key')

In [3]:
import pandas as pd
df = pd.read_csv('기자간담회.csv')[['Date','Content']]
df = df.dropna()
df

,Date,Content
0,2025-01,공 보 관 - 지금부터 2025년 1월 통화정책방향에 대한 이창용 한국은행 총재의 ...
1,2025-02,공 보 관 - 지금부터 2025년 2월 통화정책방향에 대한 이창용 한국은행 총재의 ...
2,2025-04,공 보 관 - 지금부터 2025년 4월 통화정책방향에 대한 이창용 한국은행 총재의 ...
3,2025-05,공 보 관 - 지금부터 2025년 5월 통화정책방향에 대한 이창용 한국은행 총재의 ...
5,2025-07,공 보 관 - 지금부터 2025년 7월 통화정책방향에 대한 이창용 한국은행 총재의 ...


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI  # langchain-openai 설치 필요
import os
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI 


# ✅ 2. Upstage API 클라이언트 생성
client = OpenAI(
    api_key=sola_api_key,  # Use the sola_api_key from .env
    base_url="https://api.upstage.ai/v1"
)


# ✅ 3. 요약 결과 저장할 리스트
summaries = []

# ✅ 4. 각 Content를 요약
for content in df["Content"]:
    response = client.chat.completions.create(
        model="solar-pro2",
        messages=[
            {
                "role": "system",
                "content": "한국총재가 발표한 통화정책방향을 요약해주세요."
            },
            {
                "role": "user",
                "content": content
            }
        ],
        stream=False
    )
    summary = response.choices[0].message.content.strip()
    summaries.append(summary)

# ✅ 5. 요약 결과를 DataFrame에 추가
df["default_summary"] = summaries

In [10]:
print(df['default_summary'][0])

### **2025년 1월 한국은행 통화정책방향 요약**  
**(이창용 총재 기자간담회 발표 기준)**

#### **1. 기준금리 동결 결정 (3.0% 유지)**
- **배경**:  
  - **국내외 불확실성 증대**: 미국 신정부 정책, 연준 금리 인하 속도 지연, 유럽·중국 경기 둔화, 국제유가 상승 등으로 글로벌 경제 전망 불투명.  
  - **대내적 리스크**: 국내 정치적 갈등과 소비·건설투자 부진으로 성장세 둔화.  
  - **물가 안정세 유지**: 12월 소비자물가 상승률 1.9%(석유류 가격 상승 영향), 근원물가 1.8%로 안정적 흐름 지속.  
  - **환율 변동성**: 원/달러 환율 급등과 정치적 리스크로 외환시장 불안정성 확대.  

- **금통위 의견**:  
  - 5명 찬성, 1명(신성환 위원) 반대(금리 0.25%p 인하 주장).  
  - 성장 하방 리스크에도 불구하고, **대외균형(환율, 신인도)과 정치 불확실성**을 고려해 현 수준 유지 결정.  

#### **2. 경제 전망 및 주요 리스크**
- **성장 전망**:  
  - 2024년 성장률 2.2% → **하향 조정 가능성** (정치적 리스크, 수출·소비 둔화 영향).  
  - 2025년 성장률 1.9% 전망도 하향될 수 있음.  
- **물가 전망**:  
  - 안정적 흐름 지속 예상되나, **환율 상승과 국제유가 변동성**이 상방 리스크로 작용.  
- **금융시장**:  
  - 환율 급등(원/달러 1,470원대), 주가 변동성 확대, 장기금리 하락(경기 둔화 우려 반영).  

#### **3. 향후 통화정책 방향**
- **추가 금리 인하 가능성**:  
  - 성장 하방 리스크 증대, 내수 회복 지연, 자영업자·중소기업 지원 필요성 등을 고려.  
  - **3개월 내 금리 인하 검토** (금통위원 6명 중 5명 동의).  
  - 인하 시기와 속도는 **국내 정치 상황, 미국 금리 정책, 환율 동향**에 따라 결정될 예정.  
- **유동성 

In [11]:
import pandas as pd
from openai import OpenAI  # openai==1.52.2


# ✅ 3. 프롬프트 (명확한 글쓰기 지침 포함)
system_prompt = """
당신은 글쓰기 전문가입니다. 아래 조건을 바탕으로 글을 작성해주세요:

1. Context:
- 주제는 한국은행 총재가 발표하는 통화정책방향입니다.

2. Objective:
- 독자가 장시간 발표된 내용을 요약본을 통해 편하게 인지할 수 있도록 합니다.

3. Style:
- 전문적인 글쓰기 스타일.

4. Tone:
- 격식 있고 신뢰감을 주는 톤.

5. Audience:
- 은행 직원으로 한국은행 총재의 발표 내용을 이해하고 요약할 수 있는 능력을 갖춘 사람들입니다.

6. Response:
- 가능한 한 심층 요약 글로 작성해주세요.
- 발표 내용을 주제별로 구분하여 항목별로 정리된 형태로 요약해주세요.
- 항목 제목은 발표 내용에 맞게 자유롭게 생성하되, 일관된 서식(숫자 또는 기호)을 사용해주세요.
- 발표에서 다룬 주제가 많을 경우 항목 수를 늘려도 좋습니다.

예시 형식:
1. ○○ 관련 발표 내용 요약  
2. △△ 배경 및 한국은행의 입장  
3. 향후 대응 방향
"""

# ✅ 4. 요약 결과 저장용 리스트
summaries = []

# ✅ 5. Content 열을 반복하며 요약 생성
for content in df["Content"]:
    response = client.chat.completions.create(
        model="solar-pro2",
        messages=[
            {
                "role": "system",
                "content": system_prompt.strip()
            },
            {
                "role": "user",
                "content": content.strip()
            }
        ],
        stream=False
    )
    summary = response.choices[0].message.content.strip()
    summaries.append(summary)

# ✅ 6. 결과 저장
df["sola_summary"] = summaries

In [12]:
print(df['sola_summary'][0])

### 한국은행 총재 통화정책방향 발표 심층 요약  

#### 1. **기준금리 동결 결정 배경**  
- **현재 기준금리(3%) 유지** 결정: 대내외 경제 환경의 불확실성 증대(국내 정치 리스크, 미 연준 금리 정책, 주요국 경제 전망 차이)를 고려해 신중한 접근 필요성 대두.  
- **국내 성장 둔화**: 수출 증가세 둔화, 내수 회복 지연, 건설투자 부진 등으로 2024년 성장률 전망치(1.9%) 하회 가능성 확대.  
- **물가 안정세 유지**: 12월 소비자물가 상승률 1.9%(석유류 가격 상승 영향), 근원물가 상승률 1.8%로 안정적 흐름 지속. 단, 환율 상승이 물가 상방 리스크로 작용 가능.  
- **금융시장 변동성**: 원/달러 환율 급등(1,470원대), 주가 조정 및 반등, 국고채 금리 하락(경기 둔화 우려 반영).  

#### 2. **대내외 경제 환경 평가**  
- **대외 여건**:  
  - **미국**: 고용·성장 견조, 인플레이션 둔화 지연으로 연준 금리 인하 속도 늦어질 전망.  
  - **유로존**: 정치 불확실성·제조업 부진으로 경기 회복 더딤.  
  - **중국**: 부동산 침체·보호무역 영향으로 성장 둔화 예상.  
  - **국제유가·천연가스**: 상방 리스크로 작용 가능.  
- **대내 여건**:  
  - **정치 리스크 확대**: 비상계엄 사태 이후 국내 정치 불확실성 증가로 경제 심리 위축.  
  - **성장 전망 하향 조정**: 정부 경기부양책 효과 불확실성, 미 정책 방향 등 하방 리스크 증대.  

#### 3. **통화정책 운용 방향**  
- **기준금리 추가 인하 가능성 시사**: 성장 하방 위험 확대, 내수 회복 지연에 대응 필요성 언급. 단, 인하 시기와 속도는 대내외 여건(정치 상황, 미 연준 정책, 환율 변동성) 점검 후 결정할 계획.  
- **금융중개지원대출(금융중개지원대출) 확대**: 자영업자·중소기업 지원을 위해 5조 원 추가 투입, 유동성 공급 강화.  

###

In [13]:
df.to_csv('sola_기자간담회_요약.csv', index=False)  